## We only perform our analysis on chr1 ~ chr 22, as sex chromosome expression varies between male and female, would lead to complex pipelines

In [1]:
import argparse
import pandas as pd
import numpy as np
import math
import os

In [ ]:
OUT_DIR = "/gymreklab-tscc/ydong/data/allTis/"
tisSuffix = os.listdir(OUT_DIR)

In [9]:
import os

for currTis in tisSuffix:
    try:
        os.rename(OUT_DIR+currTis+'/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_exon_reads.par_Parsed.csv',
                  OUT_DIR+currTis+'/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_exon_reads_Parsed.csv')
    except OSError:
        print(currTis)

Cervix-Endocervix
Skin-SunExposed(Lowerleg)
Thyroid
Artery-Tibial
Esophagus-Muscularis
Testis
Vagina
Esophagus-Mucosa
Skin-NotSunExposed(Suprapubic)
Artery-Aorta
Stomach
Brain-Caudate(basalganglia)
Cells-Leukemiacellline(CML)
Breast-MammaryTissue
Heart-AtrialAppendage
Brain-Cerebellum
Brain-Anteriorcingulatecortex(BA24)
Pancreas
Adipose-Subcutaneous
MinorSalivaryGland
Liver
Bladder
Esophagus-GastroesophagealJunction


In [8]:
import os
try:
    os.rename("fasdfadsfasd.txt","dfasdfas.txt")
except OSError:
    print("aaa")


aaa


In [12]:
EXONFILE = OUT_DIR + tisSuffix[1]+'/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_exon_reads_Parsed.csv'#args.exon_expr
JUNCTIONFILE =OUT_DIR + tisSuffix[1]+ '/GTEx_Analysis_2017-06-05_v8_STARv2.5.3a_junctions_Parsed.csv'#args.junction_expr
ANNOTFILE = '/storage/ydong/data/gtexRNA/gencode_gene_annotations_GRCh38.csv'#args.exprannot
    
    
annot_df_t = pd.read_csv(ANNOTFILE,index_col = 0,dtype = {"probe.chr":str})


In [13]:
exon_df_t = pd.read_csv(EXONFILE)


In [14]:
exon_df_t

,Unnamed: 0,Description,GTEX-111CU-0426-SM-5GZY1,GTEX-111FC-1326-SM-5N9D9,GTEX-1122O-0526-SM-5N9DM,GTEX-117YX-2126-SM-5GIEL,GTEX-11DXX-0726-SM-5H12X,GTEX-11EQ9-0426-SM-5A5JY,GTEX-11LCK-0126-SM-5A5M5,GTEX-11NSD-0426-SM-5N9CR,...,GTEX-ZP4G-0326-SM-4YCEF,GTEX-ZPCL-0626-SM-DNZZ6,GTEX-ZPIC-0126-SM-DNZZ9,GTEX-ZPU1-1026-SM-4YCEQ,GTEX-ZTTD-1726-SM-57WEL,GTEX-ZV6S-0826-SM-5NQ6Z,GTEX-ZVP2-0526-SM-51MSC,GTEX-ZVZP-0726-SM-59HKA,GTEX-ZYFG-0726-SM-5GIDX,GTEX-ZZPU-0126-SM-5E446
0,0,DDX11L1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.000000
1,1,DDX11L1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.118421
2,2,DDX11L1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,3,DDX11L1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,4.0,2.0,0.0,0.0,4.881579
4,4,WASH7P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657337,ENSG00000284600.1_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657338,ENSG00000284600.1_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657339,ENSG00000284600.1_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
657340,ENSG00000284600.1_6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# read in files
exon_df_t = pd.read_csv(EXONFILE,index_col = "Name")

junction_df_t = pd.read_csv(JUNCTIONFILE,index_col = "Name",dtype = {"Chr":str})

[]

In [3]:
import argparse
import pandas as pd
import numpy as np
import math




# Compute Percent Spliced In (PSI)
# PSI_df: the output matrix of psi
# exon_gene_df: exon read count matrix
# junction_gene_df: Exon-exon junction read count matrix
# annot_gene_df: annotation file for exons
def compute_gene_PSI(PSI_df,exon_gene_df,junction_gene_df,annot_gene_df,read_len):
     
    exons = list(exon_gene_df.index)
    min_reads = 10
    
    # loop through each exon
    for exon in exons:

        exon_chrom = annot_gene_df.loc[exon]['probe.chr']
        exon_start = annot_gene_df.loc[exon]['probe.start']
        exon_stop = annot_gene_df.loc[exon]['probe.stop']
        exon_len = math.fabs(exon_start - exon_stop) + 1
        exon_strand = annot_gene_df.loc[exon]['probe.strand']
        if exon_strand == '-':
            orig_start = exon_start
            exon_start = exon_stop
            exon_stop = orig_start
        
        
        #Get the A,B,C reads for PSI calculation
        B_intervals = []
        C_intervals = []
        for interval in junction_gene_df.index:
            chrom,start,stop = interval.split('_')
            chrom = chrom.split("chr")[1]
            if str(chrom) != str(exon_chrom):
                print(chrom)
                print(exon_chrom)
                print('Chromosomes aren\'t matching, something is wrong!!')

            if int(start) < exon_start and int(stop) > exon_stop:#C
                C_intervals.append(interval)
            if int(start) < exon_start and (int(stop) >= exon_start and int(stop) <= exon_stop):#B1
                B_intervals.append(interval)
            if (int(start) >= exon_start and int(start) <= exon_stop) and int(stop) > exon_stop:#B2
                B_intervals.append(interval)

        A_reads = exon_gene_df.loc[exon]
        B_reads = junction_gene_df.loc[B_intervals]
        B_reads = B_reads.iloc[:,2:].sum()

        C_reads = junction_gene_df.loc[C_intervals]
        C_reads = C_reads.iloc[:,2:].sum()

        total_reads = A_reads + B_reads + C_reads
        if len(total_reads[total_reads < min_reads])/len(total_reads) > 0.2:
            continue
        else:
            A_B_norm = (A_reads + B_reads)/(read_len + exon_len-1)
            C_norm = C_reads/(read_len-1)

            mask = (A_B_norm == 0) & (C_norm == 0)
            PSI_norm = 100*((A_B_norm)/(A_B_norm + C_norm)) if (A_B_norm.sum() + C_norm.sum()) > 0.00 else 0
            PSI_norm = PSI_norm.where(~mask,0)
            PSI_df[exon] = PSI_norm

    return PSI_df


# find out the maximum coverage sample for each individual
def max_coverage_samples(exon_df):
    'Given exon expression dataframe'
    'Take first two elements of sample name'
    'If there is duplicate sample'
    'Return list of max coverage non-dup samples'
    
    med_exon_df = pd.DataFrame(exon_df.median(axis=0))
    med_exon_df['samples'] = ["-".join(i.split('-')[:2]) for i in med_exon_df.index]
    med_exon_df['original_samples'] = med_exon_df.index
    highest_cov_samples = list(med_exon_df.groupby('samples').idxmax()[0]) 
    
    return highest_cov_samples

def filter_exons(PSI_df):
    'Given PSI dataframe'
    'Filter out exons with little variation:'
    'PSI = 100% or PSI = 0% in >90% patients'
    'Return filtered PSI'
    
    PSI_filtered_df = PSI_df.dropna(axis=1)
    exons = PSI_filtered_df.columns
    exons_to_remove = []
    num_samples = len(PSI_df.index)
    for exon in exons:
        exon_PSI = PSI_filtered_df[exon]
        num_zero = len(exon_PSI[exon_PSI == 0])
        num_hundred = len(exon_PSI[exon_PSI == 100])
        if num_zero/num_samples > 0.9 or num_hundred/num_samples > 0.9:
            exons_to_remove.append(exon)
    PSI_filtered_df = PSI_filtered_df.drop(exons_to_remove,axis=1).dropna()
    new_index = ["-".join(i.split('-')[:2]) for i in PSI_filtered_df.index]
    PSI_filtered_df.index = new_index

    return PSI_filtered_df

if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description="Compute Percent Spliced In (PSI) for any exon")
#     parser.add_argument("--exon_expr", help="File path to exon expression dataset", type=str, required=True)
#     parser.add_argument("--junction_expr", help="File path to junction expression dataset", type=str, required=True)
#     parser.add_argument("--exprannot", help="Exon expression annotation file", type=str, required=True)
#     parser.add_argument("--chrom", help="Restrict analysis to this chromosome", type=str, required=False)
#     parser.add_argument("--min_samples", help="Require data for this many samples", type=int, default=0)
#     parser.add_argument("--read_len", help="Paired end length, default is GTEx read length: 75bp ", type=int, default=75)
#     parser.add_argument("--out", help="Write data files to this file", type=str, required=True)


#     args = parser.parse_args()
    EXONFILE = '/storage/ydong/data/gtexRNA/lung/GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_exon_reads_Parsed.csv'#args.exon_expr
    JUNCTIONFILE = '/storage/ydong/data/gtexRNA/lung/GTEx_Analysis_2017-06-05_v8_STARv2.5.3a_junctions_Parsed.csv'#args.junction_expr
    ANNOTFILE = '/storage/ydong/data/gtexRNA/gencode_gene_annotations_GRCh38.csv'#args.exprannot
    
    
    # read in files
    exon_df_t = pd.read_csv(EXONFILE,index_col = "Name")
    annot_df_t = pd.read_csv(ANNOTFILE,index_col = 0,dtype = {"probe.chr":str})
    junction_df_t = pd.read_csv(JUNCTIONFILE,index_col = "Name",dtype = {"Chr":str})
    
    chrList = list(range(2,23))
    
    for CHROM in chrList:
#         CHROM = 1 #args.chrom
        MINSAMPLES = 0 #args.min_samples
        READLEN = 2*75 #args.read_len
        OUTFILE = '/storage/ydong/data/chrom/lung/chr' + str(CHROM) + '_PSI.csv'#args.out

        # format CHROM
        CHROM = str(CHROM)
        annot_df = annot_df_t[annot_df_t['probe.chr'] ==  str(CHROM)]
        junction_df =junction_df_t[junction_df_t['Description'].isin(annot_df['gene.id'])]
        junction_df = junction_df[junction_df["Chr"] == str(CHROM)]
        exon_df = exon_df_t.loc[list(annot_df['probe.id'])]

        #removing duplicated samples by taking highest coverage (in exon expression) sample

        highest_cov_samples = max_coverage_samples(exon_df) 
        exon_df = exon_df[highest_cov_samples]
        highest_cov_samples[:0] = ['Description','Chr']
        junction_df = junction_df[highest_cov_samples]
        PSI_df = pd.DataFrame(index=exon_df.columns, columns=exon_df.index)
        genes = set(annot_df['gene.id'])
        print('Chromosome: {}'.format(CHROM))
        print('Exons total: {}'.format(len(exon_df.index)))

        for gene in genes:
            exon_gene_df = exon_df.loc[exon_df.index.str.startswith(gene)]
            junction_gene_df = junction_df[junction_df['Description'] == gene]
            annot_gene_df = annot_df[annot_df['gene.id'] == gene]
            annot_gene_df.index = annot_gene_df['probe.id']
            PSI_df = compute_gene_PSI(PSI_df,
                                      exon_gene_df,
                                      junction_gene_df,
                                      annot_gene_df,
                                      READLEN)
        PSI_filtered_df = filter_exons(PSI_df.dropna(axis=1))
        print('Exons with PSI: {}\n'.format(len(PSI_filtered_df.columns)))
        PSI_filtered_df.to_csv(OUTFILE)
        print("done")

KeyError: "['Unnamed: 0'] not found in axis"

In [2]:
list(range(2,23))

[1]

In [6]:
for gene in genes:
    exon_gene_df = exon_df.loc[exon_df.index.str.startswith(gene)]
    junction_gene_df = junction_df[junction_df['Description'] == gene]
    annot_gene_df = annot_df[annot_df['gene.id'] == gene]
    annot_gene_df.index = annot_gene_df['probe.id']
    PSI_df = compute_gene_PSI(PSI_df,
                              exon_gene_df,
                              junction_gene_df,
                              annot_gene_df,
                              READLEN)
PSI_filtered_df = filter_exons(PSI_df.dropna(axis=1))
print('Exons with PSI: {}\n'.format(len(PSI_filtered_df.columns)))
PSI_filtered_df.to_csv(OUTFILE)
print("done")

Exons with PSI: 3686

done
